Objectif:
    Le but du projet est de créer un programme qui distingue les TrueNews et des FakeNews, le problème est donc un classification.
On a accès a plusieurs paramettre d'entrée mais on simplifira au maximum pour minimiser le temps de calcule. On a donc en entrée le Titre et le Texte et en sortie 0 (Fake) ou 1 (True).

Données:
    On prend un grand nombre de données (50 000 News), pour une grande précision, avec chacune un Titre, un Texte et 0 si il est faux ou 1 si il est vrai, donc uniquement les attribus les plus pertinnent à étudieer pour dire si une information est vrai ou non.
Pour améliorer la qualité, nous utiliseront 2 bases de données distinctes regroupées dans un même format.
Dans les bases que nous avons pris, il n'y a pas de donnée manquantes, mais si il y en avait eu une, on l'aurait simplement supprimé: le base de donnée est suffisement grande pour ne pas que ça impacte le résultat final.

Choix du métriques: accuracy (précision) car on est dans un problème de classification. Les résultats obtenus sont  suffisement satisfaisant (plus de 95% de réussite) pour ne pas avoir besoin d'essayer d'autres métriques.

# Initialisation DataFrame

In [1]:
import numpy as np
import pandas as pd
import re

tsize=0.25 #test size pour les train_test_split
g=0.1 #Gamma pour la classification avec SVN

chemin1="../Doc/fake_or_real_news.csv"
chemin2F = "../Doc/Fake.csv"
chemin2T = "../Doc/True.csv"

#Traitement de la 1ere BdD
dt=pd.read_csv(chemin1)
dt=dt.drop(['Unnamed: 0'], axis=1)

#Traitement de la 2nd BdD
dttmpF = pd.read_csv(chemin2F)
dttmpT = pd.read_csv(chemin2T)

dttmpF["label"] = "0"
dttmpT["label"] = "1"

dttmp = pd.concat([dttmpF,dttmpT])
dttmp = dttmp.drop(['subject', 'date'], axis=1)

for i in range(dt.shape[0]):
    if dt["label"][i] == "FAKE":
        dt["label"][i] =0;
    else:
        dt["label"][i] =1;

#Envisager de mélanger les lignes si besoin.

#Mise en commun des 2 BdD
dt = pd.concat([dt,dttmp])
print("Finish Cell")

Finish Cell


In [2]:
#Nettoyage de la dt
import string as st

def net_text(text):
    text = text.lower()
    text = re.sub('\n',' ',text)# On retire les caractères spéciaux
    text = "".join([ch for ch in text if ch not in st.punctuation]) # Retire le ponctuation
    text = re.split('\s+', text) #On sépare chaque mot pour appliquer en liste la suite
    text = [x for x in text if len(x)>2] # On retire les mots de 2 lettres, trop présents et sans grande utilité.
    text = " ".join([word for word in text]) #On recolle tout
    return text

dt['text']=dt['text'].apply(net_text)
dt['title']=dt['title'].apply(net_text)


print(dt)

Finish Cell


# Juste le texte

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

#On a besoin d'un nombre donc on va convertir le text en vecteur;


vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dt['text'].astype(str))

y=dt['label']

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=tsize)
print("Finish Cell")

Finish Cell


In [4]:
%%time
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

Arbre_decision = DecisionTreeClassifier(random_state=0, max_depth=20)
clf = Arbre_decision.fit(Xtrain, ytrain.astype(float))

ypredit = clf.predict(Xtest)

ypredit = pd.Series(ypredit)
ytest=ytest.astype(float); #pour avoir le même type

print(accuracy_score(ytest, ypredit))
print(metrics.confusion_matrix(ytest, ypredit))

0.9628386290889218
[[6361  163]
 [ 313 5972]]
Wall time: 26.8 s


In [5]:
%%time
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

KNN = KNeighborsClassifier()
clf = KNN .fit(Xtrain, ytrain.astype(float))
ypredit = clf.predict(Xtest)

ypredit = pd.Series(ypredit)
ytest=ytest.astype(float); #pour avoir le même type

print(accuracy_score(ytest, ypredit))
print(metrics.confusion_matrix(ytest, ypredit))

0.6482160980560543
[[6404  120]
 [4386 1899]]
Wall time: 42.4 s


# Juste le titre

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

#On a besoin d'un nombre donc on va convertir le text en vecteur;


vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dt['title'].astype(str))

y=dt['label']

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=tsize)
print("Finish Cell")

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

Arbre_decision = DecisionTreeClassifier(random_state=0, max_depth=20)
clf = Arbre_decision.fit(Xtrain, ytrain.astype(float))

ypredit = clf.predict(Xtest)

ypredit = pd.Series(ypredit)
ytest=ytest.astype(float); #pour avoir le même type

print(accuracy_score(ytest, ypredit))
print(metrics.confusion_matrix(ytest, ypredit))

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

KNN = KNeighborsClassifier()
clf = KNN .fit(Xtrain, ytrain.astype(float))
ypredit = clf.predict(Xtest)

ypredit = pd.Series(ypredit)
ytest=ytest.astype(float); #pour avoir le même type

print(accuracy_score(ytest, ypredit))
print(metrics.confusion_matrix(ytest, ypredit))

# Titre + texte

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

#On a besoin d'un nombre donc on va convertir le text en vecteur;


vectorizer = TfidfVectorizer()
Xtmp=dt['title'] + dt['text']
X = vectorizer.fit_transform(Xtmp);

y=dt['label']

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=tsize)
print("Finish Cell")

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

Arbre_decision = DecisionTreeClassifier(random_state=0, max_depth=20)
clf = Arbre_decision.fit(Xtrain, ytrain.astype(float))

ypredit = clf.predict(Xtest)

ypredit = pd.Series(ypredit)
ytest=ytest.astype(float); #pour avoir le même type

print(accuracy_score(ytest, ypredit))
print(metrics.confusion_matrix(ytest, ypredit))

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

KNN = KNeighborsClassifier()
clf = KNN .fit(Xtrain, ytrain.astype(float))
ypredit = clf.predict(Xtest)

ypredit = pd.Series(ypredit)
ytest=ytest.astype(float); #pour avoir le même type

print(accuracy_score(ytest, ypredit))
print(metrics.confusion_matrix(ytest, ypredit))